In [ ]:
import config, requests, json
from pathlib import Path
import pandas as pd
from datetime import datetime, date
import alpaca_trade_api as tradeapi
import os
import requests
from dotenv import load_dotenv

In [ ]:
# create a webscraper to dump a list of cryptos from coinbase into a csv file
holdings = pd.read_csv(Path("datasets/crypto_df.csv"))
holdings = holdings['symbol']
symbols = []
for symbol in holdings:
    symbols.append(symbol)
print(symbols)

In [ ]:
today = date.today()

def sub_years(years):
    today = date.today()
    try:
        return today.replace(month = today.month - years)
    except ValueError:
        return today + (date(today.month + years, 1, 1) - date(today.month, 1, 1))

start = sub_years(1).strftime('%Y-%m-%dT%H:%M:%SZ')

end = datetime.utcnow()
end = end.isoformat("T") + "Z"

print(start)
print(end)

In [ ]:
# load_dotenv()
# alpaca_api_key = os.getenv("ALPACA_API_KEY")
# alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
# alpaca_api_base_url = os.getenv("ALPACA_API_BASE_URL")
alpaca = tradeapi.REST(
    config.API_KEY,
    config.SECRET_KEY,
    api_version="v2"
)
# print(f"Alpaca Key type: {type(alpaca_api_key)}")
# print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")

In [ ]:
BTC = 'BTCUSD'
ETH = 'ETHUSD'
day_bars_url = '{}/{}/bars?exchanges=ERSX&start={}&end={}&limit=1000&timeframe=1Day'.format(config.BARS_URL, BTC, start, end)
r = requests.get(day_bars_url, headers=config.HEADERS)
print(json.dumps(r.json(), indent=4))

In [ ]:
r.json()['bars'][0]

In [ ]:
list = []
for info in r.json()['bars']:
    list.append(pd.Series(info))
test_df = pd.DataFrame(data=list)
test_df

In [ ]:
test_df.rename(columns={'t':'date', 'x':'exchange', 'o':'open', 'h':'high', 'l':'low', 'c':'close', 'v':'volume', 'n':'number_of_trades', 'vw':'volume_weighted_average_price'})


In [ ]:
stocks = ['BTCUSD', 'ETHUSD']

for stock in stocks:

    day_bars_url = '{}/{}/bars?exchanges=ERSX&start={}&end={}&limit=1000&timeframe=1Day'.format(config.BARS_URL, stock, start, end)

    r = requests.get(day_bars_url, headers=config.HEADERS)

    list = []
    for info in r.json()['bars']:
        list.append(pd.Series(info))

    test_df = pd.DataFrame(data=list)
    test_df = test_df.rename(columns={'t':'date', 'x':'exchange', 'o':'open', 'h':'high', 'l':'low', 'c':'close', 'v':'volume', 'n':'number_of_trades', 'vw':'volume_weighted_average_price'})
    test_df.to_csv('datasets/ohlc_{}.csv'.format(stock))